In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import urllib
from io import BytesIO
from concurrent.futures import ProcessPoolExecutor


We perform a web scraping to extract links to Excel files (.xlsx) from FSA payment webpage. The script identifies all the <a> tags (hyperlinks) within the webpage and filters out only those links that point to Excel files. After extracting the desired links, it constructs the absolute URLs by appending them to the main URL of the FSA website. For each Excel file URL, it retrieves the file using urllib.request.urlretrieve() and saves it locally.

In [ ]:
r = requests.get('https://www.fsa.usda.gov/news-room/efoia/electronic-reading-room/frequently-requested-information/payment-files-information/index')
r

In [ ]:
r.text

In [ ]:
soup = BeautifulSoup(r.text,'html.parser')
#soup

links = soup.find_all("a")
#links[:]


In [ ]:
xlsx_links = [link.get('href') for link in links if link.get('href') and link.get('href').endswith('.xlsx')]
main_url = "https://www.fsa.usda.gov"

for new_url in xlsx_links:
    dls = f"{main_url}{new_url}"
    filename = new_url.split('/')[-1]  # Extract the filename from the URL
    urllib.request.urlretrieve(dls, filename)
    print(f"Downloaded {filename}")

print("Download completed.")

Next, we combined multiple Excel files into a single CSV file. It begins by importing the necessary libraries, pandas and os. 
It defines a directory where the Excel files are located and lists all files with .xlsx or .xls extensions in that directory. Then, it creates an empty list to store pandas DataFrames, iterates through each Excel file in the directory, reads each file into a DataFrame using pd.read_excel(), and appends the DataFrame to the list. After reading all Excel files, it concatenates all DataFrames in the list into a single DataFrame using pd.concat(). The parameter ignore_index=True ensures that the index of the resulting DataFrame is reset. Finally, it saves the combined DataFrame to a CSV file named 'combined_data.csv' using the to_csv() method, specifying index=False to exclude the index column from the CSV file.

In [ ]:
import pandas as pd
import os

directory = r'D:/OneDrive - University of Illinois - Urbana/JupNote/FSAExcels/'
excel_files = [f for f in os.listdir(directory) if f.endswith('.xlsx') or f.endswith('.xls')]

dffs = []
for file in excel_files:
    file_path = os.path.join(directory, file)
    dff = pd.read_excel(file_path)
    dffs.append(dff)

combined_df = pd.concat(dffs, ignore_index=True)
combined_df.to_csv('combined_data.csv', index=False)


Here code reads in dataset in chunks and creates a list of Conservation Reserve Programs (CRP). Takes the unique content of 'Accounting Program Description' that has 'CRP' in their program name in each chunk and updates the list for the whole dataset. 

In [ ]:
import pandas as pd

unique_programs = set()
chonker = pd.read_csv('D:/OneDrive - University of Illinois - Urbana/JupNote/combined_data.csv', chunksize=100000, low_memory=False)


for i in chonker:
    unique_programs.update(i[i['Accounting Program Description'].str.contains('CRP ', na=False)]['Accounting Program Description'].unique())

unique_programs_list = list(unique_programs)
unique_programs_list

Here the code creates a seperate csv file only for CRP programs using the list created in the previous code.

In [ ]:
import pandas as pd
import os

# Path to the CSV file
file_path = 'D:/OneDrive - University of Illinois - Urbana/JupNote/combined_data.csv'
# Output CSV file
output_file = 'D:/OneDrive - University of Illinois - Urbana/JupNote/CRP_data.csv'

# Initialize chunk reader
chunk_iter = pd.read_csv(file_path, chunksize=1000000, low_memory=False)

# Initialize a flag to check if headers need to be written
write_header = not os.path.exists(output_file)  # True if file does not exist

# Loop through each chunk
for chunk in chunk_iter:
    # Filter the chunk
    chunk['Payment Date'] = pd.to_datetime(chunk['Payment Date'], errors='coerce')
    chunk['Payment Year'] = chunk['Payment Date'].dt.year
    filtered_chunk = chunk[chunk['Accounting Program Description'].isin(unique_programs_list)]

    # Check if filtered_chunk is not empty
    if not filtered_chunk.empty:
        # Append the filtered chunk to the output CSV file
        # Write header only if write_header is True
        filtered_chunk.to_csv(output_file, mode='a', header=write_header, index=False)
        write_header = False  # Set to False after first write
